In [1]:
import sqlite3
import sqlalchemy
import pandas as pd
import numpy as np
import json
import re
import pprint
import os
from glob import glob
import pickle

In [2]:
with open (r'data/spotify_ids.pkl', 'rb') as f:
    spotify_ids = pickle.load(f)

In [3]:
# connect to music sqlite3 database 
conn = sqlite3.connect('music.db')
c = conn.cursor()

## Connect the spotify track id to the Genius tracks 
- Read into pandas df

In [4]:
PATH = "data/lyrics/"
EXT = "*.txt"

lyric_txts = [file
             for path, subdir, files in os.walk(PATH)
             for file in glob(os.path.join(path, EXT))]

lyrics_dict = {}   # dict to store {song: lyrics} values

for fname in lyric_txts:
    fname_spl = fname.split('.txt')[0]
    fname_spl2 = fname_spl.split('\\')
    song = fname_spl2[1]
    
    with open(fname, 'r', encoding='utf8') as f:
        lyrics_dict[song] = f.read()

In [5]:
lyrics_df = (pd.DataFrame.from_dict(lyrics_dict, orient='index')).reset_index().rename(index = str, columns = {'index': 'song_name', 0: 'text'})

In [6]:
spotify_df = pd.DataFrame(spotify_ids).rename(columns = {0: 'song_name_orig', 1: 'song_name', 2: 'spotify_id'})

In [7]:
spotify_df.drop_duplicates(inplace=True)

In [8]:
print(lyrics_df.shape, spotify_df.shape)

(142, 2) (142, 3)


In [9]:
merged = lyrics_df.merge(spotify_df, how='left', on='song_name')

In [10]:
merged.shape

(142, 4)

In [11]:
# merged.head()

## Add to music db with sqlite3

In [12]:
merged.to_sql('lyrics', con=conn, index=False, if_exists='replace')

In [13]:
conn.commit()
conn.close()